In [1]:
import pandas as pd


*Working with 2012 Dataset*

In [2]:
YEAR =2012

In [24]:
# Only take 1000 entries from the dataset
case_file_1000 = pd.read_csv(f"../dataset/preprocessed/{YEAR}/case_file.csv").dropna(subset=["mark_id_char"]).head(1000)
case_file_1000.to_csv(f"../dataset/processed/{YEAR}/case_file_1000.csv", index=False)

C:\Users\vtngu\AppData\Local\Temp\ipykernel_44360\178981877.py:2: DtypeWarning: Columns (1,2,7,8,19,22,31,36,37,38,39,43,44,45,47,65,67,68,69,70,71,73,74) have mixed types. Specify dtype option on import or set low_memory=False.
  case_file_1000 = pd.read_csv(f"../dataset/preprocessed/{YEAR}/case_file.csv").dropna(subset=["mark_id_char"]).head(1000)


In [25]:
current_case_file=case_file_1000
# Convert serial_no column in case_file to a set for fast lookup
VALID_SERIALS = set(current_case_file["serial_no"])

## Mark

In [5]:
# Load case_file_cleaned_1000.csv with the required columns

# Create mark_df with serial_no and mark_id_char
mark_df = current_case_file[["serial_no", "mark_id_char"]]
mark_df["mark_length"] = mark_df["mark_id_char"].apply(lambda x: len(str(x).split()))
mark_df

,serial_no,mark_id_char,mark_length
136460,60146682,MACHINE-PISTOL,1
136974,60149828,THOMPSON,1
630084,70011210,SAMSON,1
630085,70011408,MONARCH MIXED PAINT SENOUR MANUFACTURING COMPA...,27
630086,70011709,NESTLE,1
...,...,...,...
631444,71007927,KEARNEY & FOOT D & F,6
631445,71007943,MONARCH,1
631446,71007957,VICTOR BRAND,2
631447,71007958,ANCHOR BRAND,2


## Statement and Pseudo

In [6]:

# Initialize an empty DataFrame to store the filtered data
statement_df = pd.DataFrame()
statement_chunks = pd.read_csv(f"../dataset/preprocessed/{YEAR}/statement.csv", chunksize=1000000)

# Process and filter in chunks
for chunk in statement_chunks:
    filtered_chunk = chunk[chunk["serial_no"].isin(VALID_SERIALS)]
    statement_df = pd.concat([statement_df, filtered_chunk], ignore_index=True)
    

In [7]:
pm_entries = statement_df[statement_df['statement_type_cd'].str.startswith(('PM'))]
print("Total PM entries: ", len(pm_entries))
pm_entries = pm_entries.rename(columns={"statement_text": "pseudo_mark"}).drop(columns=["statement_type_cd"])
pm_entries

Total PM entries:  112


,pseudo_mark,serial_no
21,WASHBURNS GOLD MEDAL,70013253
56,SCOTTS EMULSION PPP OF PURE COD LIVE,70019225
69,WOODWARDS CRIPE WATER,70019979
71,THE DAINTY 1ST QUALITY,70020157
74,NELSON MORRIS AND COMPANY BRAND EXTRA SUGAR CU...,70020175
...,...,...
1401,100 YARDS,71007732
1421,XF,71007897
1424,JB SMITH,71007902
1431,GH BARNETT,71007921


REMEMBER TR = TRANSLATION OF WORDS 

In [8]:
statement_df = statement_df[statement_df['statement_type_cd'].str.startswith(('GS'))]

In [9]:
duplicates = statement_df[statement_df.duplicated(subset=['serial_no'], keep=False)]
print(duplicates)

     statement_type_cd                                     statement_text  \
166             GS0181  CARBOLIC ACID, CREOSOTE, CRESYLIC ACID OR ANY ...   
167             GS0441                                          APPARATUS   
709             GS0171              TOBACCO, SNUFF, CIGARS AND CIGARETTES   
710             GS0461                                     COFFEE ABD TEA   
1182            GS0131          RODS, SHEETS, WIRE, PLATES, TUBES, INGOTS   
1183            GS0141                 CASTINGS FORMED OF METALLIC ALLOYS   

      serial_no  
166    70023441  
167    70023441  
709    70060813  
710    70060813  
1182   71004942  
1183   71004942  


## Dict Indicator

In [10]:
import nltk
from nltk.corpus import wordnet
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\vtngu\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [11]:
def dictionary_indicator(trademark):
    """Check if the trademark exists in WordNet dictionary"""
    words = str(trademark).split()  # Handle missing values
    return int(all(wordnet.synsets(word.lower()) for word in words))

In [12]:
dict_indicator_df = pd.DataFrame(mark_df["mark_id_char"].apply(dictionary_indicator))
dict_indicator_df.columns = ["dict_indicator"]
dict_indicator_df["serial_no"] = mark_df["serial_no"]
dict_indicator_df

,dict_indicator,serial_no
136460,0,60146682
136974,1,60149828
630084,1,70011210
630085,0,70011408
630086,1,70011709
...,...,...
631444,0,71007927
631445,1,71007943
631446,1,71007957
631447,1,71007958


## Create Nice Class

In [13]:
"""
import pandas as pd

# Define the Nice classes and their descriptions
nice_classes = {
    1: "Chemicals used in industry, science and photography, as well as in agriculture, horticulture and forestry; unprocessed artificial resins, unprocessed plastics; manures; fire extinguishing compositions; tempering and soldering preparations; chemical substances for preserving foodstuffs; tanning substances; adhesives used in industry.",
    2: "Paints, varnishes, lacquers; preservatives against rust and against deterioration of wood; colorants; mordants; raw natural resins; metals in foil and powder form for painters, decorators, printers and artists.",
    3: "Bleaching preparations and other substances for laundry use; cleaning, polishing, scouring and abrasive preparations; soaps; perfumery, essential oils, cosmetics, hair lotions; dentifrices.",
    4: "Industrial oils and greases; lubricants; dust absorbing, wetting and binding compositions; fuels (including motor spirit) and illuminants; candles and wicks for lighting.",
    5: "Pharmaceutical and veterinary preparations; sanitary preparations for medical purposes; dietetic food and substances adapted for medical use, food for babies; dietary supplements for humans and animals; plasters, materials for dressings; material for stopping teeth, dental wax; disinfectants; preparations for destroying vermin; fungicides, herbicides.",
    6: "Common metals and their alloys; metal building materials; transportable buildings of metal; materials of metal for railway tracks; non-electric cables and wires of common metal; ironmongery, small items of metal hardware; pipes and tubes of metal; safes; goods of common metal not included in other classes; ores.",
    7: "Machines and machine tools; motors and engines (except for land vehicles); machine coupling and transmission components (except for land vehicles); agricultural implements other than hand-operated; incubators for eggs; automatic vending machines.",
    8: "Hand tools and implements (hand-operated); cutlery; side arms; razors.",
    9: "Scientific, nautical, surveying, photographic, cinematographic, optical, weighing, measuring, signalling, checking (supervision), life-saving and teaching apparatus and instruments; apparatus and instruments for conducting, switching, transforming, accumulating, regulating or controlling electricity; apparatus for recording, transmission or reproduction of sound or images; magnetic data carriers, recording discs; compact discs, DVDs and other digital recording media; mechanisms for coin-operated apparatus; cash registers, calculating machines, data processing equipment, computers; computer software; fire-extinguishing apparatus.",
    10: "Surgical, medical, dental and veterinary apparatus and instruments; artificial limbs, eyes and teeth; orthopaedic articles; suture materials; therapeutic and assistive devices adapted for the disabled; massage apparatus; apparatus, devices and articles for nursing infants; sexual activity apparatus, devices and articles.",
    11: "Apparatus for lighting, heating, steam generating, cooking, refrigerating, drying, ventilating, water supply and sanitary purposes.",
    12: "Vehicles; apparatus for locomotion by land, air or water.",
    13: "Firearms; ammunition and projectiles; explosives; fireworks.",
    14: "Precious metals and their alloys; jewellery, precious and semi-precious stones; horological and chronometric instruments.",
    15: "Musical instruments.",
    16: "Paper and cardboard; printed matter; bookbinding material; photographs; stationery and office requisites, except furniture; adhesives for stationery or household purposes; artists' and drawing materials; paintbrushes; typewriters and office requisites (except furniture); instructional and teaching materials (except apparatus); plastic materials forsheets, films and bags for wrapping and packaging; printers' type; printing blocks.",
    17: "Unprocessed and semi-processed rubber, gutta-percha, gum, asbestos, mica and substitutes for all these materials; plastics and resins in extruded form for use in manufacture; packing, stopping and insulating materials; flexible pipes, tubes and hoses, not of metal.",
    18: "Leather and imitations of leather; animal skins, and hides; trunks and travelling bags; umbrellas and parasols; walking sticks; whips, harness and saddlery; collars, leashes and clothing for animals.",
    19: "Building materials (non-metallic); non-metallic rigid pipes for building; asphalt, pitch and bitumen; non-metallic transportable buildings; monuments, not of metal.",
    20: "Furniture, mirrors, picture frames; containers, not of metal, for storage or transport; unworked or semi-worked bone, horn, ivory, whalebone or mother-of-pearl; shells; meerschaum; yellow amber.",
    21: "Household or kitchen utensils and containers; combs and sponges; brushes, (except paintbrushes); brush-making materials; articles for cleaning purposes; steelwool; unworked or semi-worked glass, (except building glass used in building); glassware, porcelain and earthenware.",
    22: "Ropes and string; nets; tents, awnings, and tarpaulins; awnings of textile or synthetic materials; sails; sacks for the transport and storage of materials in bulk; padding, cushioning and stuffing materials, (except of paper, cardboard, rubber or plastics); raw fibrous textile materials and substitutes therefor.",
    23: "Yarns and threads, for textile use.",
    24: "Textiles and substitutes for textiles; bed covers; table covers; household linen; curtains of textile or plastic.",
    25: "Clothing, footwear, headgear.",
    26: "Lace and embroidery, ribbons and braid; buttons, hooks and eyes, pins and needles; artificial flowers; hair decorations; false hair.",
    27: "Carpets, rugs, mats and matting, linoleum and other materials for covering existing floors; wall hangings (non-textile).",
    28: "Games, toys and playthings; video game apparatus; gymnastic and sporting articles; decorations for Christmas trees.",
    29: "Meat, fish, poultry and game; meat extracts; preserved, frozen, dried and cooked fruits and vegetables; jellies, jams, compotes; eggs; milk and milk products; edible oils and fats"}
 

"""

'\nimport pandas as pd\n\n# Define the Nice classes and their descriptions\nnice_classes = {\n    1: "Chemicals used in industry, science and photography, as well as in agriculture, horticulture and forestry; unprocessed artificial resins, unprocessed plastics; manures; fire extinguishing compositions; tempering and soldering preparations; chemical substances for preserving foodstuffs; tanning substances; adhesives used in industry.",\n    2: "Paints, varnishes, lacquers; preservatives against rust and against deterioration of wood; colorants; mordants; raw natural resins; metals in foil and powder form for painters, decorators, printers and artists.",\n    3: "Bleaching preparations and other substances for laundry use; cleaning, polishing, scouring and abrasive preparations; soaps; perfumery, essential oils, cosmetics, hair lotions; dentifrices.",\n    4: "Industrial oils and greases; lubricants; dust absorbing, wetting and binding compositions; fuels (including motor spirit) and ill

In [14]:
"""
nice_classes.update({
    30: "Coffee, tea, cocoa and artificial coffee; rice; tapioca and sago; flour and preparations made from cereals; bread, pastries and confectionery; edible ices; sugar, honey, treacle; yeast, baking powder; salt; mustard; vinegar, sauces (condiments); spices; ice.",
    31: "Raw and unprocessed agricultural, aquacultural, horticultural and forestry products; raw and unprocessed grains and seeds; fresh fruits and vegetables, fresh herbs; natural plants and flowers; bulbs, seedlings and seeds for planting; live animals; foodstuffs and beverages for animals; malt.",
    32: "Beers; mineral and aerated waters and other non-alcoholic beverages; fruit beverages and fruit juices; syrups and other preparations for making beverages.",
    33: "Alcoholic beverages (except beers).",
    34: "Tobacco; smokers' articles; matches.",
    35: "Advertising; business management; business administration; office functions.",
    36: "Insurance; financial affairs; monetary affairs; real estate affairs.",
    37: "Building construction; repair; installation services.",
    38: "Telecommunications services.",
    39: "Transport; packaging and storage of goods; travel arrangement.",
    40: "Treatment of materials. recycling of waste and trash; air purification and treatment of water;printing services; food and drink preservation.",
    41: "Education; providing of training; entertainment; sporting and cultural activities.",
    42: "Scientific and technological services and research and design relating thereto; industrial analysis, industrial research and industrial design services; quality control and authentication services; design and development of computer hardware and software.",
    43: "Services for providing food and drink; temporary accommodation.",
    44: "Medical services; veterinary services; hygienic and beauty care for human beings or animals; agriculture, aquaculture, horticulture and forestry services.",
    45: "Legal services; security services for the physical protection of tangible property and individuals; personal and social services rendered by others to meet the needs of individuals."
})
"""

'\nnice_classes.update({\n    30: "Coffee, tea, cocoa and artificial coffee; rice; tapioca and sago; flour and preparations made from cereals; bread, pastries and confectionery; edible ices; sugar, honey, treacle; yeast, baking powder; salt; mustard; vinegar, sauces (condiments); spices; ice.",\n    31: "Raw and unprocessed agricultural, aquacultural, horticultural and forestry products; raw and unprocessed grains and seeds; fresh fruits and vegetables, fresh herbs; natural plants and flowers; bulbs, seedlings and seeds for planting; live animals; foodstuffs and beverages for animals; malt.",\n    32: "Beers; mineral and aerated waters and other non-alcoholic beverages; fruit beverages and fruit juices; syrups and other preparations for making beverages.",\n    33: "Alcoholic beverages (except beers).",\n    34: "Tobacco; smokers\' articles; matches.",\n    35: "Advertising; business management; business administration; office functions.",\n    36: "Insurance; financial affairs; moneta

In [15]:
nice_df = pd.read_csv("../dataset/processed/nice_class.csv", usecols=["nice_class_code", "nice_class_description"])

print(nice_df.head())  

   nice_class_code                             nice_class_description
0                1  Chemicals used in industry, science and photog...
1                2  Paints, varnishes, lacquers; preservatives aga...
2                3  Bleaching preparations and other substances fo...
3                4  Industrial oils and greases; lubricants; dust ...
4                5  Pharmaceutical and veterinary preparations; sa...


In [16]:
# Ensure all columns are numeric
intl_class_df = pd.DataFrame(columns=["intl_class_cd", "class_id", "serial_no"], dtype=int)
intl_class_chunks = pd.read_csv(f"../dataset/preprocessed/{YEAR}/intl_class.csv", chunksize=1000000)

# Process and filter in chunks
for chunk in intl_class_chunks:
    filtered_chunk = chunk[chunk["serial_no"].isin(VALID_SERIALS)]
    intl_class_df = pd.concat([intl_class_df, filtered_chunk], ignore_index=True)


# Find entries in VALID_SERIALS but not in intl_class_df
missing_serials = VALID_SERIALS - set(intl_class_df["serial_no"])
# Print the mark_id_char for the missing serial numbers
missing_marks = current_case_file[current_case_file["serial_no"].isin(missing_serials)]
print("Mark ID Characters for missing serial numbers:")
print(missing_marks[["serial_no", "mark_id_char"]])


Mark ID Characters for missing serial numbers:
        serial_no                                    mark_id_char
630123   70018310                     THE WESTERN BOILER COMPOUND
630158   70021903                                       KUTTYHUNK
630194   70023649                                       EXCELSIOR
630199   70023794  GRAY'S COMPOUND TONIC FORMULA DR. JOHN P. GRAY
630232   70025628                                   IVERTON MILLS
630247   70025989                                     BULL'S HEAD
630306   70028774                                  STANDARD BRAND
630307   70028858                                             VTW
630651   70217993                                            KING
630889   70438036                                 SHELTER PACKAGE
631060   71002013                                      FISH BRAND
631248   71004920                                      BUMSTEAD'S
631250   71004940                                      BEE DESIGN
631352   71006555            

In [17]:
intl_class_df = intl_class_df.apply(pd.to_numeric, errors='coerce')
intl_class_df = intl_class_df.drop_duplicates(subset=['serial_no'])
intl_class_df

,intl_class_cd,class_id,serial_no
0,9,1,60146682
1,9,2,60149828
2,22,3,70011210
3,2,4,70011408
4,29,5,70011709
...,...,...,...
1032,6,1086,71007920
1033,8,1087,71007921
1034,8,1088,71007927
1035,25,1089,71007943


In [18]:
nice_class_df = pd.DataFrame(columns=["serial_no","nice_class_code","nice_class_description"], dtype=int)
nice_class_df = intl_class_df[intl_class_df["serial_no"].isin(VALID_SERIALS)].merge(
    nice_df, left_on="intl_class_cd", right_on="nice_class_code", how="left"
)[["serial_no", "nice_class_code", "nice_class_description"]]
nice_class_df

,serial_no,nice_class_code,nice_class_description
0,60146682,9,"Scientific, nautical, surveying, photographic,..."
1,60149828,9,"Scientific, nautical, surveying, photographic,..."
2,70011210,22,"Ropes and string; nets; tents, awnings, and ta..."
3,70011408,2,"Paints, varnishes, lacquers; preservatives aga..."
4,70011709,29,"Meat, fish, poultry and game; meat extracts; p..."
...,...,...,...
979,71007920,6,Common metals and their alloys; metal building...
980,71007921,8,Hand tools and implements (hand-operated); cut...
981,71007927,8,Hand tools and implements (hand-operated); cut...
982,71007943,25,"Clothing, footwear, headgear."


## Input

In [19]:
merged_df = pd.DataFrame(columns=["serial_no", "mark_id_char" ,"statement_text", "nice_class_code ", "nice_class_description","dict_indicator","pseudo_mark"])
merged_df = nice_class_df[["serial_no", "nice_class_code", "nice_class_description"]]

In [20]:
merged_df = merged_df.merge(mark_df, on="serial_no", how="left")
merged_df = merged_df.merge(statement_df, on="serial_no", how="left")

merged_df = merged_df.merge(dict_indicator_df, on="serial_no", how="left")

merged_df = merged_df.merge(pm_entries, on="serial_no", how="left", suffixes=("", "_pm"))
merged_df["pseudo_mark"].fillna("", inplace=True)

merged_df.drop(columns=["statement_type_cd"], inplace=True)
merged_df


C:\Users\vtngu\AppData\Local\Temp\ipykernel_44360\4056937329.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merged_df["pseudo_mark"].fillna("", inplace=True)


,serial_no,nice_class_code,nice_class_description,mark_id_char,mark_length,statement_text,dict_indicator,pseudo_mark
0,60146682,9,"Scientific, nautical, surveying, photographic,...",MACHINE-PISTOL,1,pistols,0,
1,60149828,9,"Scientific, nautical, surveying, photographic,...",THOMPSON,1,"shotguns, pistols, rifles, machine-guns, subma...",1,
2,70011210,22,"Ropes and string; nets; tents, awnings, and ta...",SAMSON,1,"CORDS, LINES, [ TWINES, ] AND ROPES",1,
3,70011408,2,"Paints, varnishes, lacquers; preservatives aga...",MONARCH MIXED PAINT SENOUR MANUFACTURING COMPA...,27,MIXED PAINTS,0,
4,70011709,29,"Meat, fish, poultry and game; meat extracts; p...",NESTLE,1,CONDENSED MILK,1,
...,...,...,...,...,...,...,...,...
989,71007920,6,Common metals and their alloys; metal building...,MONARCH,1,WIRE FENCE,1,
990,71007921,8,Hand tools and implements (hand-operated); cut...,BLACK WORKS G. H. BARNETT,5,FILES AND RASPS,0,GH BARNETT
991,71007927,8,Hand tools and implements (hand-operated); cut...,KEARNEY & FOOT D & F,6,FILES AND RASPS,0,KF
992,71007943,25,"Clothing, footwear, headgear.",MONARCH,1,HOSIERY,1,


In [21]:
final_data_df = merged_df.dropna(subset=['nice_class_code'])

In [22]:
final_data_df["formatted_text"] = final_data_df.apply(
    lambda row: f"[CLS]{row['mark_id_char']} [SEP] {row['statement_text']} [SEP] {'mark present in Wordnet' if row['dict_indicator'] else 'mark absent in Wordnet'} [SEP] Mark length is {row['mark_length']} [SEP] NICE category is {row['nice_class_code']} [SEP] {row['nice_class_description']} [SEP] Pseudo mark is {row['pseudo_mark']} [CLS]",
    axis=1
)
final_data_df.drop(columns=["nice_class_description"], inplace=True)
final_data_df.to_csv(f"../dataset/processed/{YEAR}/final_data.csv", index=False)


In [23]:
input_df = final_data_df[['serial_no','formatted_text']]
input_df.to_csv(f"../dataset/processed/{YEAR}/input.csv",index=False)